# What I want to do with this
##                                                                EDA setup
I would like to take the average household statistics from the range of bedrooms, bathrooms, zipcodes etc and adjust the average of all the zeros that show up in those ranges with the averages of those areas. Possibly change the waterfront views that are set to zero in that are near the lat's and long's as well from zeros and put them as ones to better represent the data a little more accurately. After taking a look at that and making sure thats acceptable I will start by setting a function to graph the data on a map.(Don't take too long on this as it is a decent representation of the information it is also not needed).
##                                                    Interesting representation
Possible representation of the data could be through zipcode and bin them in with each other and set them out as zones as well. Could take a look at the 'Areas' as a way to make sense of the data and the pricing of each. could also set up a function to pull specific information for the conditions of what a particular pearson wants as well as a budget. Taking note from both House Hunters and DIY shows on what it is that could be done with deemable 'Fixer Uppers' with a certain budget. Set that function towards anything below their budget and whether they want to fix a place up or not. If time alloted I will use the map made earlier to graph the places and show where those places are adjesent to important places like hospitals, schools, possibly shopping centers, etc. Could also figure a way to find out if the zones have crime rates etc around them and deem if it is a safe neighborhood.

##                                                   Statistical representation
Train a model to take in the account of stuff the person is looking at and adjust the amount of money they would spend in "upgrading" their home and how much they would expect in return for their investments if they were to sell their house, refinance etc and if it would be deemed acceptible if they want to flip the house how much they would be able to sell later on.                                            



# Exploring the data
First things first, checking and fixing up the data. I am looking for anything that doesn't belong like NaN's and irrelevent data

In [258]:
# !pip install arcgis
import pandas as pd
import arcgis
from arcgis.gis import GIS
from arcgis.features import GeoAccessor,GeoSeriesAccessor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('data/kc_house_data.csv')
gis = GIS()

,X,Y,NAME,ABB_NAME,ADDRESS,ZIPCODE,DISTRICT,FEATUREDES
0,-122.152303,47.608461,Sammamish High School,Sammamish,100 140th Ave SE,98005,BELLEVUE,High
1,-122.205415,47.768906,Saint Brendan Catholic School,Saint Brendan,10049 NE 195th St,98011,NORTHSHORE,Elementary
2,-122.348454,47.525265,Highland Park Elementary School,Highland Park,1012 SW Trenton St,98106,SEATTLE,Elementary
3,-122.200170,47.667245,Lakeview Elementary School,Lakeview,10400 NE 68th St,98033,LAKE WASHINGTON,Elementary
4,-122.294727,47.708042,Nathan Hale High School,Nathan Hale,10750 30th Ave NE,98125,SEATTLE,High
...,...,...,...,...,...,...,...,...
637,-122.218847,47.759860,Bothell High School,Bothell,9130 NE 180th St,98011,NORTHSHORE,High
638,-122.318017,47.694273,The Perkins School,Perkins,9005 Roosevelt Way NE,98115,SEATTLE,Elementary
639,-122.225130,47.318565,Green River College Auburn Center,Green R Coll - Auburn,1221 D St NE,98002,NaN,College or University
640,-122.138094,47.264257,Muckleshoot Tribal School,Muckleshoot Tribal,15209 SE 376th St,98092,AUBURN,K-12


## Checking the dataframe
Taking a look at what the dataframe comprises of and taking a look at what irrelevent data is there

In [118]:
df.head()



,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [97]:
display(df.isna().sum())
display(df.max())
df.min()
df.price.astype(int)


id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

id               9900000190
date               9/9/2014
price               7.7e+06
bedrooms                 33
bathrooms                 8
sqft_living           13540
sqft_lot            1651359
floors                  3.5
waterfront                1
view                      4
condition                 5
grade                    13
sqft_above             9410
sqft_basement             ?
yr_built               2015
yr_renovated           2015
zipcode               98199
lat                 47.7776
long               -121.315
sqft_living15          6210
sqft_lot15           871200
dtype: object

0        221900
1        538000
2        180000
3        604000
4        510000
          ...  
21592    360000
21593    400000
21594    402101
21595    400000
21596    325000
Name: price, Length: 21597, dtype: int32

## Outliers
it seems that there are a few missing data and some outliers. The max shows a bedroom capacity of 33, will take a look at the here.

In [98]:
df[df['bedrooms'] == 33]
# wouldn't be realistic for a house with 33 bedrooms to only have a sqft_living of 1620 and only 1 3/4 bathrooms so it looks like a typo
# will adjust to 3 
df[df['bedrooms'] == 33] = df[df['bedrooms'] == 33].replace(33,3)
df['sqft_basement'] = df['sqft_basement'].replace('?','0').astype(float)
df['waterfront'] = df['waterfront'].fillna(0)
df['view'] = df['view'].fillna(0)
# should remove id, lat,long from dataframe to work with median 
df1 = df.drop('id',1).drop('lat',1).drop('long',1)


In [99]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     21597 non-null  float64
 9   view           21597 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  float64
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [100]:
df.head()

,X,Y,NAME,ABB_NAME,ADDRESS,ZIPCODE,DISTRICT,FEATUREDES
0,-122.152303,47.608461,Sammamish High School,Sammamish,100 140th Ave SE,98005,BELLEVUE,High
1,-122.205415,47.768906,Saint Brendan Catholic School,Saint Brendan,10049 NE 195th St,98011,NORTHSHORE,Elementary
2,-122.348454,47.525265,Highland Park Elementary School,Highland Park,1012 SW Trenton St,98106,SEATTLE,Elementary
3,-122.200170,47.667245,Lakeview Elementary School,Lakeview,10400 NE 68th St,98033,LAKE WASHINGTON,Elementary
4,-122.294727,47.708042,Nathan Hale High School,Nathan Hale,10750 30th Ave NE,98125,SEATTLE,High


In [101]:
school_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   X           642 non-null    float64
 1   Y           642 non-null    float64
 2   NAME        642 non-null    object 
 3   ABB_NAME    642 non-null    object 
 4   ADDRESS     642 non-null    object 
 5   ZIPCODE     642 non-null    int64  
 6   DISTRICT    641 non-null    object 
 7   FEATUREDES  642 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 40.2+ KB


In [102]:
df1.median()

price            450000.00
bedrooms              3.00
bathrooms             2.25
sqft_living        1910.00
sqft_lot           7618.00
floors                1.50
waterfront            0.00
view                  0.00
condition             3.00
grade                 7.00
sqft_above         1560.00
sqft_basement         0.00
yr_built           1975.00
yr_renovated          0.00
zipcode           98065.00
sqft_living15      1840.00
sqft_lot15         7620.00
dtype: float64

# Pulling extra data from outside sources
The normal housing dataset looks to be in good shape now. What we are looking at now is common things that potential home buyers are going to look for when purchasing houses. Examples are: expecting parents looking for a good school close by, grocery stores that they can have easy access to, public transport around the area and possibly some other recreational activites like parks.

In [213]:
school_df = pd.read_csv('data/Schools.csv')

def gis_content():
    item = gis.content.get('ff9e4774ee8641f48cccac97dca753db')
    flayer = item.layers[0]
    food = flayer.query(where = "ESITE >= 0")
    item = gis.content.get('3f263039314d44cc93384fe1f4229796#data')
    flayer = item.layers[0]
    public = flayer.query(where = "ZIPCODE > 0").sdf.copy()
    item = gis.content.get('0dfe37d2a68545a699b999804354dacf')
    flayer = item.layers[0]
    private = flayer.query(where = "STATE = 'WA'").sdf.copy()
    
    #Editing data for later use
    public_df = public.drop(['FID','PIN','SCH_CLASS','CODE','FEATURE_ID','ESITE','MAJOR','MINOR','SHAPE'],1)
    private_df = private.drop(['FID','NCESID','ZIP4','VAL_METHOD','VAL_DATE','SHELTER_ID','COUNTRY','SOURCE','SOURCEDATE','NAICS_CODE','FT_TEACHER','START_GRAD','END_GRADE','COUNTYFIPS','TYPE','STATUS','VAL_METHOD','VAL_DATE'],1)
    food_df = food.drop(['RECORD_ID','FACILITY_NAME','CHAIN_NAME','CHAIN_ESTABLISHMENT','SITE_ADDRESS','ABB_NAME','ESITE','FEATURE_ID',1])
    return (food_df,public_df,private_df)


# Editing extra data

<module 'arcgis.mapping' from 'C:\\Users\\Thomas\\anaconda3\\lib\\site-packages\\arcgis\\mapping\\__init__.py'>

In [214]:
food,public,private = gis_content()

In [215]:
public.head()

,NAME,ABB_NAME,ADDRESS,ZIPCODE,LONG_CEN,LAT_CEN,DISTRICT,WEBSITE
0,Glacier Park School,Glacier Park,23700 SE 280th St,98038,-122.025583,47.351607,TAHOMA,http://www.tahoma.wednet.edu/SubSite/GPES/inde...
1,Rosa Parks School,Rosa Parks,22845 NE Cedar Park Crescent,98053,-122.034854,47.684738,LAKE WASHINGTON,http://publicportal.lwsd.org/schools/rosaparks...
2,Evergreen School,Evergreen,15201 Meridian Ave N,98133,-122.335278,47.738587,SHORELINE,http://www.evergreenschool.org/about-evergreen...
3,Seahawks Academy 6-8,Seahawks Academy 6-8,1600 S Columbian Way,98108,-122.312085,47.564131,SEATTLE,none
4,Family Learning Center,Family Learning Center,2315 173rd Ave NE,98052,-122.109820,47.630440,LAKE WASHINGTON,http://washington.schooltree.org/public/Family...


In [103]:
def schoolzone(budget,schools):
    """This function is to find the schools related to the zipcode of the houses and display them on the map
       to find which is the closes schools to the houses"""
    schoolzone = []
    for zipcode in budget.columns('zipcode'):
        if schools[schools['zipcode']] == zipcode:
            schoolzone.append(zipcode)
            
    return schoolzone

# Adding in markets
want to add markets into the final mapping product. need to figure out how I am going to make each layer show up when wanted through either a layer difference or through a different means. try making one map for everything in one go

In [267]:
def mapping(df):
    """Taking in a dataframe to assign the budget through all fields. Takes schools into account as well. If not looking for schools
        just enter through"""
    budget = float(input('What is your budget: '))
    schools = input('Are you interested in schools? ')
    budget_df = df.loc[df.price<=budget].copy()
    budget_df.is_copy = None
    pdx_map = gis.map('Seattle,WA')
    if (schools == 'Yes') or (schools == 'yes') and (budget is not None):
        school_sdf = pd.DataFrame.spatial.from_xy(public,'LONG_CEN','LAT_CEN')
        school_sdf.spatial.plot(map_widget = pdx_map,renderer_type = "u",symbol_type = 'simple',symbol_style = 'o',title = 'Schools', 
                      col='ZIPCODE',
                      cmap='spring',  # matplotlib color map
                    alpha=0.7,
                    size = .5, 
                    outline_color=[0,0,0,0])
        
    pdx_map.basemap = 'dark-gray'
    
    price = df[df['price']<= budget] 

    df.price.sort_values(ascending= True)
    data_sdf = pd.DataFrame.spatial.from_xy(budget_df, 'long','lat')
    
    
       
    data_sdf.spatial.plot(map_widget=pdx_map, renderer_type = "c",symbol_type = 'simple',symbol_style='d',
                    title='Pricing of houses',
                    col='price',
                    cmap='winter',  # matplotlib color map
                    alpha=0.7,
                    size = .5,
                    outline_color=[0,0,0,0]
                     )
    

    display(pdx_map)
# took legend out since the unique renderer_type makes it hard to assign general information through it. 
# Will work on getting that working so the distance from houses shows schools locally by zipcode.
# sns.barplot(x = 'price',y = 'sqft_living',data = budget_df)

In [266]:
mapping(df)

What is your budget: 100000
Are you interested in schools? yes


MapView(layout=Layout(height='400px', width='100%'))

,POPULATION,LATITUDE,LONGITUDE,ENROLLMENT
count,496.000000,496.000000,496.000000,496.000000
mean,149.334677,47.453419,-121.569997,136.570565
std,205.499581,0.684067,1.684867,192.323893
min,2.000000,45.470048,-124.197574,1.000000
25%,27.000000,47.245392,-122.428481,22.750000
50%,82.000000,47.615994,-122.285712,72.000000
75%,189.000000,47.736218,-122.032594,174.000000
max,1705.000000,48.982110,-117.051147,1679.000000


KeyError: "['FID' 'NCESID' 'ZIP4' 'VAL_METHOD' 'VAL_DATE' 'SHELTER_ID' 'COUNTRY'\n 'SOURCE' 'SOURCEDATE' 'NAICS_CODE' 'FT_TEACHER' 'START_GRAD' 'END_GRADE'\n 'COUNTYFIPS' 'TYPE' 'STATUS' 'VAL_METHOD' 'VAL_DATE'] not found in axis"

In [51]:
from arcgis.features import FeatureLayer
fs_url = 'https://www.arcgis.com/home/item.html?id=c7a17b7ad3ec44b7ae64796dca691d72'
KC_housing = FeatureLayer(fs_url)

In [55]:
KC_housing

<FeatureLayer url:"https://www.arcgis.com/home/item.html?id=c7a17b7ad3ec44b7ae64796dca691d72">

In [62]:
pdx_map = gis.map('Seattle,WA')
school_sdf = pd.DataFrame.spatial.from_xy(housing_info,'LAT','LON')
school_sdf.spatial.plot(map_widget=pdx_map, renderer_type = "c", title='Pricing of houses',
                    col='price',
                    cmap='winter',  # matplotlib color map
                    alpha=0.7,
                     
                     outline_color=[0,0,0,0]
                     )
pdx_map

KeyError: 'LAT'

In [113]:
private_df

,NAME,ADDRESS,CITY,STATE,ZIP,TELEPHONE,POPULATION,COUNTY,LATITUDE,LONGITUDE,NAICS_DESC,WEBSITE,LEVEL_,ENROLLMENT,SHAPE
0,HOLY FAMILY ELEMENTARY SCHOOL,1002 CHESTNUT ST,CLARKSTON,WA,99403,(509) 758-6621,97,ASOTIN,46.411282,-117.051147,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,89,"{""x"": -13030074.072459644, ""y"": 5846504.160099..."
1,BETHLEHEM LUTHERAN SCHOOL,2505 W 27TH AVE,KENNEWICK,WA,99337,(509) 582-5624,185,BENTON,46.195222,-119.126914,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,174,"{""x"": -13261147.35571504, ""y"": 5811689.0603052..."
2,CHILDREN'S GARDEN MONTESSORI,2440 GARLICK BLVD,RICHLAND,WA,99354,(509) 375-1638,45,BENTON,46.319501,-119.279506,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,42,"{""x"": -13278133.858861264, ""y"": 5831698.087068..."
3,LIBERTY CHRISTIAN SCHOOL,2200 WILLIAMS BLVD,RICHLAND,WA,99354,(509) 946-0602,392,BENTON,46.285753,-119.300263,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,3,362,"{""x"": -13280444.516195828, ""y"": 5826260.211634..."
4,CHILDRENS GATE MONTESSORI SCHOOL,1425 CHERRY ST,WENATCHEE,WA,98801,(509) 886-0533,8,CHELAN,47.415093,-120.336569,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,5,"{""x"": -13395805.618508115, ""y"": 6010092.962589..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,THE MADRONE SCHOOL,10020 166TH AVE NE,REDMOND,WA,98052,(425) 269-1833,17,KING,47.741449,-122.040276,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,3,7,"{""x"": -13585461.377696253, ""y"": 6063948.499051..."
492,THE MOOSE PROJECT,801 E 2ND AVE STE 3,SPOKANE,WA,99202,(509) 534-1402,10,SPOKANE,47.654983,-117.398542,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,8,"{""x"": -13068745.86443973, ""y"": 6049646.9885231..."
493,THREE POINTS ELEMENTARY SCHOOL CAMPUS,7800 NE 28TH ST,MEDINA,WA,98039,(425) 454-3977,241,KING,47.636068,-122.235169,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,223,"{""x"": -13607156.759423066, ""y"": 6046521.689881..."
494,THREE RIVERS CHRISTIAN SCHOOL,2610 OCEAN BEACH HWY,LONGVIEW,WA,98632,(360) 423-4510,271,COWLITZ,46.148023,-122.950136,ELEMENTARY AND SECONDARY SCHOOLS,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,3,250,"{""x"": -13686746.55842117, ""y"": 5804101.8957813..."
